# Cisco DevNet - RestAPI

* 什么是 API ？


![API](assert/api.png)

In [1]:
import time

class SERVER():
    def __init__(self):
        self.url = 'yeslab.com'
        self.data = { 'route':['to_192.168.1.0', 'to_172.21.29.0', 'to_10.123.11.0']}
        self.keys = list(self.data.keys())
        self.announce()
        
    def announce(self):
        print("My url is {}".format(self.url))
        print("My service is {}".format(self.keys[0]))
        
    def response(self, receive):
        msg = receive()
        print("【SERVER】API请求处理中...")
        time.sleep(2)
        if msg == self.url + '/' + self.keys[0]:
            return self.data['route']
    
server = SERVER()

My url is yeslab.com
My service is route


In [2]:
@server.response
def client_send():
    print("【CLIENT】API请求发送中...")
    time.sleep(2)
    return "yeslab.com/route"

print(client_send)

【CLIENT】API请求发送中...
【SERVER】API请求处理中...
['to_192.168.1.0', 'to_172.21.29.0', 'to_10.123.11.0']


![API2](assert/api2.png)

* REST is "just another use for the HTTP Protocol." A REST API service sends all requests and responses over TCP/TLS connections using text.
* REST is an API framework built on HTTP, and the interface points are often for web services. When you combine REST and API, you see a simple service interface that enables applications or people to use the HTTP protocol to request objects or information.

![HTTP](assert/http.png)

### HTTP 请求方法
* POST 新建
* GET 读取
* PUT 更新
* PATCH 更新
* DELETE 删除

![restapi](assert/restapi.png)

![url](assert/url.png)

## APIC-EM APIs
https://github.com/CiscoDevNet/apicem-apis-with-python-sample-codes

### 1. 输入远端设备信息

In [6]:
import requests
import json
from helper import apicem

requests.packages.urllib3.disable_warnings() # Disable warnings

# APIC-EM IP, modify these parameters if you are using your own APIC-EM
apicem_ip = "sandboxapicem.cisco.com"
username = "devnetuser"
password = "Cisco123!"
version = "v1"

### 2. 拿到访问授权

In [7]:
# JSONhttps://sandboxapic.cisco.com/ input
r_json = {
    "username": username,
    "password": password
}

# POST ticket API URL
post_url = "https://"+apicem_ip+"/api/"+version+"/ticket"

# All APIC-EM REST API request and response content type is JSON.
headers = {'content-type': 'application/json'}

# Make request and get response - "resp" is the response of this request
resp = requests.post(post_url, json.dumps(r_json), headers=headers,verify=False)
print ("Request Status: ",resp.status_code)

# Get the json-encoded content from response
response_json = resp.json()
print ("\nRaw response from POST ticket request:\n",resp.text)
# Not that easy to read the raw response, so try the formatted print out

# Pretty print the raw response
print ("\nPretty print response:\n",json.dumps(response_json,indent=4))

Request Status:  200

Raw response from POST ticket request:
 {"response":{"serviceTicket":"ST-31846-S0A1maMTNYfv5SduyoIc-cas","idleTimeout":1800,"sessionTimeout":21600},"version":"1.0"}

Pretty print response:
 {
    "response": {
        "serviceTicket": "ST-31846-S0A1maMTNYfv5SduyoIc-cas",
        "idleTimeout": 1800,
        "sessionTimeout": 21600
    },
    "version": "1.0"
}


### 3. GET 查看用户信息

In [9]:
try:
    resp = apicem.get(apicem_ip, version, username, password, api="user")
    response_json = resp.json() # Get the json-encoded content from response
    print (json.dumps(response_json,indent=4),'\n') # Convert "response_json" object to a JSON formatted string and print it out
except:
    print ("Something wrong with GET /user request")
    sys.exit()

# Parsing raw response to list out all users and their role
for item in response_json["response"]:
    for item1 in item["authorization"]:
        print ("User \'%s\', role is the %s."%(item["username"],(item1["role"])[5:]))


Executing GET 'https://sandboxapicem.cisco.com/api/v1/user'

GET 'user' Status:  200 

{
    "response": [
        {
            "username": "devnetuser",
            "authorization": [
                {
                    "scope": "ALL",
                    "role": "ROLE_POLICY_ADMIN"
                }
            ],
            "authSource": "internal"
        }
    ],
    "version": "1.0"
} 

User 'devnetuser', role is the POLICY_ADMIN.


### 3.1 GET 角色列表失败

In [10]:
try:
    resp = apicem.get(apicem_ip, version, username, password, api="user/user")
    response_json = resp.json() # Get the json-encoded content from response
    print (json.dumps(response_json,indent=4),'\n') # Convert "response_json" object to a JSON formatted string and print it out
except:
    print ("Something wrong with GET /user request")
    sys.exit()


Executing GET 'https://sandboxapicem.cisco.com/api/v1/user/user'

GET 'user/user' Status:  401 

{
    "response": {
        "errorCode": "Unauthorized",
        "message": "You do not have permissions to access this resource",
        "detail": "User does not have sufficient privileges to access API"
    },
    "version": "1.0"
} 



### 4. POST 添加用户失败

In [11]:
add_user_data = {
                  "username": "yeslab_coder",
                  "password": "yeslab12345",
                  "authSource": "internal",
                  "authorization": [
                    {
                      "scope": "ALL",
                      "role": "ROLE_POLICY_ADMIN"
                    }
                  ],
                  "oldPassword": "yeslab12345"
                 }

resp = apicem.post(apicem_ip, version, username, password, api="user", data=add_user_data)


Executing POST 'https://sandboxapicem.cisco.com/api/v1/user'

POST 'user' Status:  403 



### 5. POST 添加策略标签成功

In [34]:
add_tag_data = {"policyTag": "Yeslab"}
policy_resp = apicem.post(apicem_ip, version, username, password, api="policy/tag", data=add_tag_data)


Executing POST 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

POST 'policy/tag' Status:  202 



In [36]:
policy_tag_result = apicem.get(apicem_ip, version, username, password, api="policy/tag")
for tag in policy_tag_result.json()['response']:
    print(tag['policyTag'])


Executing GET 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

GET 'policy/tag' Status:  200 

luisga
Vijoy_Added_this
Student_Added_This
lala
UTCN
Bryan_gonz
vanakkad
mahovich
GMR
RemoteSite
lpj
devent_tag
RemoteSite1
duno
test
TheBigNetwork
new_policy_tag


### 6. DELETE 删除不顺眼的标签

In [32]:
del_tag_data = {"policyTag": "GMR"}
policy_resp = apicem.delete(apicem_ip, version, username, password, api="policy/tag", params=del_tag_data)


Executing DELETE 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

DELETE 'policy/tag' Status:  202 



In [35]:
policy_tag_result = apicem.get(apicem_ip, version, username, password, api="policy/tag")
for tag in policy_tag_result.json()['response']:
    print(tag['policyTag'])


Executing GET 'https://sandboxapicem.cisco.com/api/v1/policy/tag'

GET 'policy/tag' Status:  200 

luisga
Vijoy_Added_this
Student_Added_This
lala
UTCN
Bryan_gonz
vanakkad
mahovich
GMR
RemoteSite
lpj
devent_tag
RemoteSite1
duno
test
TheBigNetwork
new_policy_tag
